In [1]:
import sys
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image as keras_image
from keras.utils import np_utils
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Model, model_from_json
from keras.optimizers import SGD

Using TensorFlow backend.


In [9]:
IMAGES_PATH = "../inputs/images/"
LABELS_PATH = "../inputs/labels/"

WEIGHTS_PATH = "../outputs/weights/"
NUM_CLASSES  = 8 

In [3]:
def read_inputs():
    X = []
    Y = []
    for root, dirs, files, in os.walk(LABELS_PATH):
        for file in files:
            if file.endswith(".txt") and not file.startswith("."):
                images_path = root.replace(LABELS_PATH, IMAGES_PATH)
                images      = list(filter(lambda x: not x.startswith("."), os.listdir(images_path))) # keep images that doesn't start with .

                label_file = open(os.path.join(root,file),"r")
                label = int(float(label_file.readline().strip("\n")))

                #print (images_path + ' ==> ' + str(label))

                images.sort() # only get the last four (maybe ?)
                for img in images[-4:]:
                    tmp = keras_image.load_img(images_path + "/" + img , target_size=(64, 64))
                    tmp = keras_image.img_to_array(tmp) / float(255)
                    X.append(tmp)
                    Y.append(label)
    return np.asarray(X), np_utils.to_categorical(np.asarray(Y), NUM_CLASSES)

In [4]:
def simple_model():
    model = Sequential()
    model.add(Flatten(input_shape=(64, 64, 3)))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=8, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    return model

In [5]:
def VGGNET16():
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape = (64,64,3)) 
    
    stacked = Flatten()(vgg16.output)
    stacked = Dense(64, activation='relu')(stacked)
    predictions = Dense(8, activation='softmax')(stacked)
    model = Model(input=vgg16.input, output=predictions)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model  

In [6]:
X, Y = read_inputs()

In [14]:
model = VGGNET16()

/anaconda3/envs/emotion-classification/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  import sys


In [ ]:
#checkpoint = ModelCheckpoint(WEIGHTS_PATH + "vgg16-weights.h5", verbose=1, save_best_only=True)
#callbacks_list = [checkpoint]
model.fit(X, Y, epochs=100, validation_split=0.25, verbose=1, shuffle=True)

Train on 981 samples, validate on 327 samples
Epoch 1/100
981/981 [==============================] - 66s 67ms/step - loss: 1.5634 - acc: 0.3986 - val_loss: 1.3096 - val_acc: 0.5199
Epoch 2/100
981/981 [==============================] - 65s 67ms/step - loss: 1.2942 - acc: 0.5341 - val_loss: 1.6248 - val_acc: 0.4251
Epoch 3/100
960/981 [============================>.] - ETA: 1s - loss: 0.7977 - acc: 0.7208